In [5]:
import torch
from torchsummary import summary

class EvaluationMetrics:
    def __init__(self, 
                 accuracy_list=[], 
                 f1_score_list=[], 
                 precision_list=[], 
                 recall_list=[], 
                 cross_entropy_loss_list=[], 
                 l1_loss_list=[], 
                 mse_loss_list=[], 
                 full_epos=None, 
                 epos_now=0, 
                 path_to_log_file=None, 
                 path_to_image_file_=None, 
                 path_to_log_directory="./", 
                 torchsummary_summary=None, 
                 optimizer_type=None, 
                 list_params=[], 
                 current_loss=None):
        self.Accuracy_list = accuracy_list
        self.F1_score_list = f1_score_list
        self.Precision_list = precision_list
        self.Recall_list = recall_list
        self.CrossEntropyLoss_list = cross_entropy_loss_list
        self.L1Loss_list = l1_loss_list
        self.MSELoss_list = mse_loss_list
        self.Full_epos = full_epos
        self.Epos_now = epos_now
        self.Path_to_log_file = path_to_log_file
        self.Path_to_image_file_ = path_to_image_file_
        self.Path_to_log_directory = path_to_log_directory
        self.Torchsummary_summary = torchsummary_summary
        self.OptimizerType = optimizer_type
        self.ListParms = list_params
        self.CurrentLoss = current_loss

    def set_summary(self, model, input_size):
        self.Torchsummary_summary = summary(model, input_size=input_size)

    def set_optimizer_type(self, optimizer_type):
        self.OptimizerType = optimizer_type

    def add_accuracy(self, accuracy):
        self.Accuracy_list.append(accuracy)

    def add_f1_score(self, f1_score):
        self.F1_score_list.append(f1_score)

    def add_precision(self, precision):
        self.Precision_list.append(precision)

    def add_recall(self, recall):
        self.Recall_list.append(recall)

    def add_cross_entropy_loss(self, loss):
        self.CrossEntropyLoss_list.append(loss)

    def add_l1_loss(self, loss):
        self.L1Loss_list.append(loss)

    def add_mse_loss(self, loss):
        self.MSELoss_list.append(loss)

    def set_full_epos(self, full_epos):
        self.Full_epos = full_epos

    def set_epos_now(self, epos_now):
        self.Epos_now = epos_now

    def set_path_to_log_file(self, path):
        self.Path_to_log_file = path

    def set_path_to_image_file(self, path):
        self.Path_to_image_file_ = path

    def set_path_to_log_directory(self, path):
        self.Path_to_log_directory = path

    def set_list_params(self, params):
        self.ListParms = params

    def set_current_loss(self, loss):
        self.CurrentLoss = loss



In [6]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

class EvaluationMetrics:
    def __init__(self):
        self.Accuracy_list = []
        self.CrossEntropyLoss_list = []

    def add_accuracy(self, accuracy):
        self.Accuracy_list.append(accuracy)

    def add_cross_entropy_loss(self, loss):
        self.CrossEntropyLoss_list.append(loss)

# Определение простой модели нейронной сети
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
        self.fc = nn.Linear(7*7*32, 10)  # 10 classes for MNIST

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Загрузка и предобработка датасета MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)

# Инициализация модели, функции потерь и метрик оценки
model = SimpleModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
evaluation_metrics = EvaluationMetrics()

# Обучение модели
num_epochs = 5
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Оценка точности и потерь и добавление в метрики
        _, predicted = torch.max(outputs.data, 1)
        total = labels.size(0)
        correct = (predicted == labels).sum().item()
        accuracy = correct / total
        evaluation_metrics.add_accuracy(accuracy)
        evaluation_metrics.add_cross_entropy_loss(loss.item())

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item(), (accuracy * 100)))

print("Обучение завершено!")



Epoch [1/5], Step [100/600], Loss: 0.3382, Accuracy: 89.00%
Epoch [1/5], Step [200/600], Loss: 0.0887, Accuracy: 97.00%
Epoch [1/5], Step [300/600], Loss: 0.0470, Accuracy: 98.00%
Epoch [1/5], Step [400/600], Loss: 0.0959, Accuracy: 96.00%
Epoch [1/5], Step [500/600], Loss: 0.0426, Accuracy: 99.00%
Epoch [1/5], Step [600/600], Loss: 0.0303, Accuracy: 99.00%
Epoch [2/5], Step [100/600], Loss: 0.0776, Accuracy: 96.00%
Epoch [2/5], Step [200/600], Loss: 0.0686, Accuracy: 98.00%
Epoch [2/5], Step [300/600], Loss: 0.0215, Accuracy: 99.00%
Epoch [2/5], Step [400/600], Loss: 0.0443, Accuracy: 100.00%
Epoch [2/5], Step [500/600], Loss: 0.0491, Accuracy: 99.00%
Epoch [2/5], Step [600/600], Loss: 0.0650, Accuracy: 97.00%
Epoch [3/5], Step [100/600], Loss: 0.0396, Accuracy: 99.00%
Epoch [3/5], Step [200/600], Loss: 0.0521, Accuracy: 98.00%
Epoch [3/5], Step [300/600], Loss: 0.0277, Accuracy: 99.00%
Epoch [3/5], Step [400/600], Loss: 0.0107, Accuracy: 100.00%
Epoch [3/5], Step [500/600], Loss: 0.0